In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler_market_data = StandardScaler()
fit_scaler = scaler_market_data.fit(df_market_data)
transform_scaler = fit_scaler.transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
scaled_data = pd.DataFrame(transform_scaler, columns = df_market_data.columns)

# Copy the crypto names from the original data
scaled_data["coin_id"]=df_market_data.index
# Set the coinid column as index
scaled_data = scaled_data.set_index("coin_id")

# Display sample data
scaled_data.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k_values:
    km_model = KMeans(n_clusters=i, random_state=1)
    km_model.fit(scaled_data)
    inertia.append(km_model.inertia_)



In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve = {"k": k_values, "inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_curve)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
km_elbow = elbow_df.hvplot.line(x = "k", y = "inertia", title = "Elbow Curve")
km_elbow

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k would be 4 since this value on our graoh is the most prominent point. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
Kmeans_model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the scaled data
Kmeans_model.fit(scaled_data)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predictions = Kmeans_model.predict(scaled_data)

# Print the resulting array of cluster values.
print(predictions)

In [ ]:
# Create a copy of the DataFrame
scaled_data_copy = scaled_data.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
scaled_data_copy["predicted_clusters"]=predictions

# Display sample data
scaled_data_copy.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
price_change_plot = scaled_data_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by='predicted_clusters',
    title="Price Change Percentage",
    hover_cols='coin_id'
)
price_change_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca =PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_model = [pca.fit_transform(scaled_data_copy)]
# View the first five rows of the DataFrame. 
pca_model[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.890 or 89%

In [ ]:
# Create a new DataFrame with the PCA data.
pca_data = pd.DataFrame(pca_model, columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original data
pca_data["coin_id"] = df_market_data.index

# Set the coinid column as index
pca_data=pca_data.set_index("coin_id")

# Display sample data
pca_data.head(10)

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values= range(1,11)

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_values:
    km = KMeans(n_clusters=i, random_state=1)
    km.fit(pca_model)
    inertia.append(km.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_df = pd.DataFrame({"k":k_values, "inertia":inertia})
# Create a DataFrame with the data to plot the Elbow curve


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca = elbow_df.hvplot.line(x = "k", y = "inertia", title = "PCA")
elbow_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, this k value does not differ from the orignal data

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model_pca = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the PCA data
model_pca.fit(pca_data)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_predictions = model_pca.predict(pca_data)
# Print the resulting array of cluster values.
print(pca_predictions)

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_predictions_df = pca_model.copy

# Add a new column to the DataFrame with the predicted clusters
pca_predictions_df["clusters"] = pca_predictions

pca_predictions_df.head()


In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pc_clusters = pca_predictions_df.hvplot.scatter(
    x="PC1", y="PC2", by="clusters", title="PCA Clusters", hover_cols='coin_id'
)
pc_clusters

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves

(elbow_df + elbow_pca).cols(1)

In [ ]:
# Composite plot to contrast the clusters
(price_change_plot + pc_clusters).cols(1)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After seeing that both elbow curves yeilded the same results, PCA data allows us to see more pinpointed data. Using fewer features allows for a clearer view of the outliers in a dataset.